In [ ]:
## PARTE 1 Desafio Manipulación y transformación de datos vicente soto 

import pandas as pd

# Cargando los archivos y creando los dataframes

df_incidents = pd.read_pickle("incidents.pkl")
df_officers = pd.read_pickle("officers.pkl")
df_subjects = pd.read_pickle("subjects.pkl")

# Uniendo las 3 tablas

df_merge1 = pd.merge(df_incidents, df_officers, on="case_number" , suffixes=("_inc", "_off"))
df_final = pd.merge(df_merge1, df_subjects, on="case_number", suffixes=("", "_subj"))

# Verificar si hay filas duplicadas y eliminandolas

duplicados = df_final.duplicated().sum()
print("Filas duplicadas:", duplicados)

df_final = df_final.drop_duplicates()

# Conteo de genero F en DataFrame

conteo_generos = df_final["gender_subj"].value_counts()

print(conteo_generos)

print("Cantidad de sujetos de género F:", conteo_generos.get("F", 0))

# Filtrar sujetos F 

casos_mujer = df_final[df_final["gender_subj"] == "F"]["case_number"].nunique()
print("Número de casos con al menos una mujer:", casos_mujer)

df_final.columns.tolist()


# Tabla pivote: género del oficial (gender) vs. género del sujeto (gender_subj)

pivot_generos = pd.pivot_table(
    df_final,
    index="gender",          
    columns="gender_subj",  
    values="case_number",    
    aggfunc="count"
)

pivot_generos


In [45]:
## Parte 2 

import pandas as pd
import numpy as np

# Cargando los archivos y creando los dataframes
df_jobs = pd.read_csv("Cleaned_DS_Jobs.csv")

# Remplazando valores
valores_nulos = ["na", "NA", -1, "0", "-1", "null", "n/a", "N/A", "NULL"]

# Reemplazar en todo el DataFrame
df_jobs.replace(valores_nulos, np.nan, inplace=True)

# Eliminando todas las filas que tengan valores nan
df_jobs = df_jobs.dropna()

# Creando columna de salario mínimo
df_jobs["Salario Min"] = df_jobs["Salary Estimate"].apply(lambda x: int(x.split("-")[0].strip()))

# Creando columna de salario máximo
df_jobs["Salario Max"] = df_jobs["Salary Estimate"].apply(lambda x: int(x.split("-")[1].strip()))


# Diccionario de mapeo
mapa_size = {
    "10000+ employees": "Mega Empresas",
    "5001 to 10000 employees": "Grandes Empresas",
    "1001 to 5000 employees": "Medianas Empresas",
    "201 to 500 employees": "Pequeñas Empresas",
    "51 to 200 employees": "Pequeñas Grandes Empresas",
    "501 to 1000 employees": "Microempresas",
    "Unknown": "Empresas sin Información"
}

# Reemplazando valores en la columna Size
df_jobs["Size"] = df_jobs["Size"].replace(mapa_size)


# Creando tabla pivote con promedio de salario mínimo y máximo por tamaño de empresa
pivot_salarios = pd.pivot_table(
    df_jobs,
    index="Size",                        
    values=["Salario Min", "Salario Max"],  
    aggfunc="mean"                       
)

pivot_salarios

,Salario Max,Salario Min
Size,,
Empresas sin Información,110.500000,73.000000
Grandes Empresas,138.875000,92.125000
Medianas Empresas,137.461538,93.923077
Mega Empresas,151.111111,97.888889
Microempresas,146.235294,100.176471
Pequeñas Empresas,141.142857,93.571429
Pequeñas Grandes Empresas,137.666667,100.666667
